In [10]:
import torch
import torchvision
import os
import torchvision.transforms as transforms
from torchvision import datasets, models
import numpy as np
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torch.utils.data.sampler import SubsetRandomSampler

In [11]:
class DiabeticRetinopathyDataset(Dataset):
    def __init__(self, base_dir, transform=None):
        self.base_dir = base_dir
        self.transform = transform
        self.data = []
        categories = ['No_DR', 'Mild', 'Moderate', 'Severe', 'Proliferate_DR']

        for idx, category in enumerate(categories):
            path = os.path.join(base_dir, category)
            images = os.listdir(path)[:1000]  # Take only 1000 images per category
            for image in images:
                self.data.append((os.path.join(path, image), idx))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label


In [12]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Assuming 'base_dir' is your dataset directory path
base_dir = 'data/gaussian_filtered_images/gaussian_filtered_images'  # Update this path
dataset = DiabeticRetinopathyDataset(base_dir=base_dir, transform=transform)

# Splitting the dataset into train and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [13]:
model = models.vgg16(pretrained=True)

# Freeze model parameters
for param in model.features.parameters():
    param.requires_grad = False

# Modify the classifier
num_features = model.classifier[6].in_features
features = list(model.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, 5)]) # Add our layer with 5 outputs
model.classifier = nn.Sequential(*features) # Replace the model classifier

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


/Users/pavankumardharmoju/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/pavankumardharmoju/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)

In [15]:
epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Epoch {epoch+1}, Train Loss: {running_loss / len(train_loader)}, Validation Loss: {val_loss / len(val_loader)}, Validation Accuracy: {100 * correct / total}%')


Epoch 1, Train Loss: 1.3180769317679935, Validation Loss: 0.8306207160154978, Validation Accuracy: 68.53146853146853%
Epoch 2, Train Loss: 1.0767698751555548, Validation Loss: 0.8598845733536614, Validation Accuracy: 71.15384615384616%
Epoch 3, Train Loss: 1.1011504009366035, Validation Loss: 0.969506926006741, Validation Accuracy: 67.13286713286713%
Epoch 4, Train Loss: 1.049792666402128, Validation Loss: 0.8800554407967461, Validation Accuracy: 71.5034965034965%
Epoch 5, Train Loss: 1.0093780640098784, Validation Loss: 0.8800578051143222, Validation Accuracy: 70.1048951048951%
Epoch 6, Train Loss: 0.9827726926240656, Validation Loss: 0.780158422059483, Validation Accuracy: 70.27972027972028%
Epoch 7, Train Loss: 0.8756736864646276, Validation Loss: 0.8600136107868619, Validation Accuracy: 72.9020979020979%
Epoch 8, Train Loss: 0.8790322182079157, Validation Loss: 0.8996715380085839, Validation Accuracy: 71.32867132867133%
Epoch 9, Train Loss: 0.8878432847559452, Validation Loss: 0.90